In [ ]:
import re
import nltk
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,roc_curve
import seaborn as sn
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
stemmer=PorterStemmer()

nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df = pd.read_excel(r"/content/drive/MyDrive/cleaned_agr.xlsx")

df.head()

,Unnamed: 0,Comments,level
0,0,courage said sonuyou dadagiri well muslims stand,oag
1,1,private sector banks like public icici banks w...,nag
2,2,adhere this pakistan question,oag
3,3,fake comprised gathered rascals muslims know i...,oag
4,4,happens course slaughterso stop leather manufa...,nag


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
categories = df['level'].unique()
print(categories)

['oag' 'nag' 'cag']


In [ ]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(df['Comments'].values.astype('U'))
final_features.shape

(12000, 8497)

*Random Forests*

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
import pickle

In [ ]:
#first we split our dataset into testing and training set:
# this block is to split the dataset into training and testing set 
X = df['Comments']
Y = df['level']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1700)),
                     ('clf', RandomForestClassifier())])
# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train.values.astype('U'), y_train.values.astype('U'))
with open('RandomForest.pickle', 'wb') as f:
    pickle.dump(model, f)
ytest = np.array(y_test)


In [ ]:
# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test.values.astype('U'))))
print(confusion_matrix(ytest, model.predict(X_test.values.astype('U'))))

              precision    recall  f1-score   support

         cag       0.44      0.42      0.43      1077
         nag       0.57      0.69      0.62      1270
         oag       0.49      0.32      0.39       653

    accuracy                           0.52      3000
   macro avg       0.50      0.48      0.48      3000
weighted avg       0.51      0.52      0.50      3000

[[457 479 141]
 [311 876  83]
 [260 181 212]]


*KNN*

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X = df['Comments']
Y = df['level']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1700)),
                     ('clf', KNeighborsClassifier())])
# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train.values.astype('U'), y_train.values.astype('U'))
with open('KNN.pickle', 'wb') as f:
    pickle.dump(model, f)
ytest = np.array(y_test)

In [ ]:
# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test.values.astype('U'))))
print(confusion_matrix(ytest, model.predict(X_test.values.astype('U'))))

              precision    recall  f1-score   support

         cag       0.43      0.43      0.43      1113
         nag       0.51      0.72      0.59      1240
         oag       0.61      0.13      0.22       647

    accuracy                           0.48      3000
   macro avg       0.52      0.43      0.41      3000
weighted avg       0.50      0.48      0.45      3000

[[482 597  34]
 [331 888  21]
 [296 266  85]]


*svm*

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [ ]:
X = df['Comments']
Y = df['level']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1700)),
                     ('clf', StandardScaler(with_mean=False)),
                     ('svc', SVC(gamma='auto'))])
# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train.values.astype('U'), y_train.values.astype('U'))
with open('SVM.pickle', 'wb') as f:
    pickle.dump(model, f)
ytest = np.array(y_test)

In [ ]:
# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test.values.astype('U'))))
print(confusion_matrix(ytest, model.predict(X_test.values.astype('U'))))

              precision    recall  f1-score   support

         cag       0.47      0.41      0.43      1064
         nag       0.59      0.73      0.65      1276
         oag       0.47      0.35      0.40       660

    accuracy                           0.53      3000
   macro avg       0.51      0.49      0.49      3000
weighted avg       0.52      0.53      0.52      3000

[[433 468 163]
 [250 930  96]
 [248 183 229]]


**multinomial naive bayes**

In [ ]:
X = df['Comments']
Y = df['level']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1400)),
                     ('clf', MultinomialNB())])
# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train.values.astype('U'), y_train.values.astype('U'))
with open('mnb.pickle', 'wb') as f:
    pickle.dump(model, f)
ytest = np.array(y_test)

In [ ]:
# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test.values.astype('U'))))
conf_matrix = confusion_matrix(ytest, model.predict(X_test.values.astype('U')))
print(confusion_matrix(ytest, model.predict(X_test.values.astype('U'))))

              precision    recall  f1-score   support

         cag       0.45      0.49      0.47      1045
         nag       0.58      0.78      0.66      1248
         oag       0.67      0.18      0.29       707

    accuracy                           0.54      3000
   macro avg       0.57      0.48      0.47      3000
weighted avg       0.56      0.54      0.51      3000

[[509 494  42]
 [252 974  22]
 [358 220 129]]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

**ADA BOOST**

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import time

In [ ]:
data = df.copy()

In [ ]:
data["len"] = [len(text) for text in data["Comments"].values]

data.groupby("level")["len"].mean()

level
cag    83.319734
nag    74.453687
oag    96.956280
Name: len, dtype: float64

In [ ]:
data["level"].value_counts()

nag    4977
cag    4216
oag    2699
Name: level, dtype: int64

In [ ]:
y = []
for level in data["level"]:
    
    if level=="nag":
        y.append(0)
        
    elif level=="oag":
        y.append(2)
        
    elif level=="cag":
        y.append(1)

y = np.asarray(y)

In [ ]:


# First, we need a vectorizer object
vectorizer = TfidfVectorizer(max_features=4000)
# This means just consider most used 4000 words

start = time.time()

x = vectorizer.fit_transform(cleanedText).toarray()

process_time = round(time.time()-start,2)

print("Vectorizing cleaned text using TF-IDF approach took {} seconds".format(process_time))

x.shape

NameError: ignored